<a href="https://colab.research.google.com/github/aburkov/theLMbook/blob/main/GRPO_Qwen_0_5_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import numpy as np
import random
import torch

# 设置随机种子
def set_random_seed(seed: int = 42):
    """
    Set the random seed for reproducibility across Python, NumPy, and PyTorch.

    Parameters:
        seed (int): The seed value to use.
    """
    # Set the seed for Python's built-in random module
    random.seed(seed)

    # Set the seed for NumPy
    np.random.seed(seed)

    # Set the seed for PyTorch
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    # Ensure deterministic behavior in cuDNN (may impact performance)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(42)

import os
# os.environ["WANDB_API_KEY"] = "093293c0bb1c42de69dedbc504a6f69e29b322b3"
# os.environ["WANDB_PROJECT"] = "GRPO-Qwen-7b-Instruct"


import os
import hashlib
import tarfile
import requests
import re
import wandb
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq,DataCollatorForLanguageModeling, TrainerCallback, PreTrainedTokenizerBase, default_data_collator, DataCollatorWithPadding
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig, get_peft_model

from swanlab.integration.transformers import SwanLabCallback

[2025-04-10 03:50:09,030] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/hpclp/anaconda3/envs/deepseek2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/hpclp/anaconda3/envs/deepseek2/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


INFO 04-10 03:50:12 __init__.py:183] Automatically detected platform cuda.
WARNING 04-10 03:50:12 cuda.py:323] Detected different devices in the system: 
WARNING 04-10 03:50:12 cuda.py:323] NVIDIA A800 80GB PCIe
WARNING 04-10 03:50:12 cuda.py:323] NVIDIA A800-SXM4-80GB
WARNING 04-10 03:50:12 cuda.py:323] NVIDIA A800 80GB PCIe
WARNING 04-10 03:50:12 cuda.py:323] NVIDIA A800 80GB PCIe
WARNING 04-10 03:50:12 cuda.py:323] Please make sure to set `CUDA_DEVICE_ORDER=PCI_BUS_ID` to avoid unexpected behavior.


In [2]:
# 从生成的答案中提取最终结果
def extract_xml_answer(text):
    # 尝试匹配闭合标签格式（如 <答案>...</答案>）
    match = re.search(r"<答案>(.*?)</答案>", text, re.DOTALL)
    if not match:
        # 匹配非闭合标签格式（如 <答案>...）
        match = re.search(r"<答案>(.*)", text, re.DOTALL)
    
    if match:
        answer_content = match.group(1).strip()
        # 提取第一个出现的字母（不区分大小写）
        letter_match = re.search(r"[A-Za-z]", answer_content)
        if letter_match:
            return letter_match.group().upper()  # 统一返回大写字母
        else:
            print("未找到有效的字母选项")
            return ''
    else:
        print("未找到<答案>标签")
        return ''

In [3]:
SYSTEM_PROMPT = """你是一个逻辑推理专家，擅长对问题进行元认知分析，并生成答案。"""
def get_SFT_dataset(data_files, split="train"):
    # """Load and prepare the GSM8K dataset for training."""
    data = load_dataset('json', data_files=data_files)[split]
    formatted_data = []

    for example in data:
        formatted_example = {
            "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": example['instruction']},
            {"role": "assistant", "content": example['output']}
            ]
        }
        formatted_data.append(formatted_example)

    return formatted_data


# def get_datasets(data_files) -> Dataset:
#     # 加载 JSON 文件
#     data = load_dataset('json', data_files=data_files)

#     return data['train']


In [4]:
sft_dataset = get_SFT_dataset('/home/hpclp/disk/q/study/0-毕业/微调数据生成/500-750tokens.json')

In [5]:
# len(sft_dataset)

In [6]:
sft_dataset[0]

{'messages': [{'role': 'system', 'content': '你是一个逻辑推理专家，擅长对问题进行元认知分析，并生成答案。'},
  {'role': 'user',
   'content': '你看到一副画面，画面内容为: \n##\n问题：\n每个人都喜欢话剧团。演员喜欢它是因为每晚扮演不同的角色可以减少他们的无聊程度。舞台工作人员喜欢它是因为每晚更换布景意味着更多的加班和, ,  . 剧院经理喜欢它，因为如果选择反映观众需求的戏剧来制作，大多数演出都会产生大量收入。因此，显然，更多的剧院应该转向保留剧目。\n题目：\n如果以下哪一项为真，上述论点将得到加强？\nA. 在话剧团中，必须为多部作品提供存储空间。\nB. 在话剧团中，一个每季开始时都需要大量的资金投入。\nC. 在话剧团中，可以根据观众的需求重新安排演出时间。\nD. 在话剧团中，一些演员每晚都在换角色，这很难做到。掌握他们扮演的所有角色。\n。\n##\n你需要从第三视角旁观角度记录，描述你看到现象。\n根据画面内容和现象描述，获取有潜在价值的信息。\n根据已有信息，进行详细且全面的长思考，生成尽可能长的内容。\n最终输出格式如下：\n<现象描述>\n...\n</现象描述>\n<潜在信息>\n...\n</潜在信息>\n<长思考>\n...\n</长思考>\n<答案>\n...(在<答案>后只输出选项，不输出其他内容，如只输出A)'},
  {'role': 'assistant',
   'content': '<现象描述>\n画面中呈现了一个关于话剧团的论点，指出演员、舞台工作人员和剧院经理各自喜欢保留剧目制度的原因，并由此得出更多剧院应该转向保留剧目的结论。问题要求选出能加强该论点的选项。\n</现象描述>\n\n<潜在信息>\n1. 论点强调保留剧目对剧院的多方利益：演员的多样化角色、舞台工作人员的额外加班、经理的收入提升。  \n2. 论证的关键逻辑是保留剧目的优势能够满足各方的需求，从而推断其应被推广。  \n3. 需要加强的可能是保留剧目与观众需求、收入增长之间的直接联系，或排除潜在障碍。\n</潜在信息>\n\n<长思考>\n1. **论点核心分析**：  \n   论证试图通过列举演员、工作人员和经理的偏好，证明保留剧目的制度对剧院有利。隐含的前提

In [ ]:
class ChatDataCollator:
    def __init__(self, tokenizer: PreTrainedTokenizerBase,max_length: int = 2048):
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __call__(self, batch):
        inputs_list = []
        attention_list = []
        labels_list = []
        for example in batch:

            inputs = self.tokenizer.apply_chat_template(
                example['messages'][:-1],
                add_generation_prompt=True,
                tokenize=False
            )
            inputs = inputs + example['messages'][-1]['content'] + '<|im_end|>\n'
            inputs = self.tokenizer(inputs, truncation=True, max_length=self.max_length, padding=True)  # , return_tensors='pt'
            '''
            $^-^$：直接return_tensors='pt'和
            input_ids = torch.tensor(inputs["input_ids"])
            attention_mask = torch.tensor(inputs['attention_mask'])
            居然是有区别的，还没有研究是什么区别

            直接return_tensors='pt'会增强一个维度！如原本为[1,2,3]会变成[[1,2,3]]
            '''
            input_ids = torch.tensor(inputs["input_ids"])
            attention_mask = torch.tensor(inputs['attention_mask'])
            inputs_list.append(input_ids)
            attention_list.append(attention_mask)
            labels_list.append(input_ids)

        # pad_sequence()用于对一个批次（batch）中的序列数据进行填充（padding），以使它们具有相同的长度
        inputs_padded = pad_sequence(inputs_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_padded = pad_sequence(attention_list, batch_first=True, padding_value=0)
        labels_padded = pad_sequence(labels_list, batch_first=True, padding_value=-100)
        return {"input_ids": inputs_padded, "attention_mask": attention_padded, "labels": labels_padded}

### 奖励函数

In [ ]:
def main():
    # Determine the device: use GPU if available, else fallback to CPU.
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # print(f"Using device: {device}")

    # swanlab_callback = SwanLabCallback(
    #     workspace='godblack',
    #     project='study',
    #     experiment_name='qwen2.5',
    # )
    # callbacks = [swanlab_callback]

    # Model configuration.
    model_name = "/home/hpclp/disk/q/models/Qwen2.5-0.5B-Instruct"
    output_dir = "/home/hpclp/disk/q/study/大模型从零构建/微调学习/output"

    # Load the pre-trained model on CPU first, then move to GPU.
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2",
        # device_map='auto'
    )
    model = model.to(device)

    # lora_config = LoraConfig(
    #     r=16,  # LoRA 的秩
    #     lora_alpha=32,  # LoRA 的 alpha 参数
    #     target_modules=[
    #             "q_proj", "k_proj", "v_proj", "o_proj", # 如果 OOM 内存不足，可以移除 QKVO
    #             "gate_proj", "up_proj", "down_proj",
    #         ],  # 目标模块，通常是注意力机制中的查询和值投影
    #     lora_dropout=0.1,  # LoRA 的 dropout 率
    #     bias="none",  # 是否调整偏置
    #     task_type="CAUSAL_LM",  # 任务类型
    # )

    # model = get_peft_model(model, lora_config)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    ###########################
    # Step 1: SFT Fine-Tuning #
    ###########################
    sft_dataset = get_SFT_dataset('/home/hpclp/disk/q/study/0-毕业/微调数据生成/500-750tokens.json')

    sft_training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=1,
        per_device_train_batch_size=2,
        learning_rate=5e-5,
        save_steps=200,
        logging_steps=5,
        bf16=True,
        weight_decay=0.01,
        # gradient_checkpointing=True,
        remove_unused_columns=False,    # 如果模型有未使用的列，则移除它们以节省内存；需要设置为False
        report_to="none"    # 不设置的话默认使用所有日志工具，尤其是wandb
    )

    print("\nStarting SFT fine-tuning...")
    sft_trainer = Trainer(
        model=model,
        # tokenizer=tokenizer,
        args=sft_training_args,
        train_dataset=sft_dataset,
        # # padding='longest'根据批次中最长的样本长度进行填充//padding='max_length'即填充到指定的最大长度。
        # data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding='longest')
        data_collator=ChatDataCollator(tokenizer),
        # callbacks=callbacks
    )
    sft_trainer.train()

    sft_trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)



if __name__ == "__main__":
    main()


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.



Starting SFT fine-tuning...


Step,Training Loss
500,1.771900
1000,1.692700
